In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from PIL import Image

from skimage.transform import resize
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.models as models

In [ ]:
root = "/kaggle/input/compsiimmodified"

df = pd.read_csv(f"{root}/train.csv", index_col=0, dtype={"label": "category"})

df["boxes"] = df["boxes"].apply(lambda x: eval(x))

# Remove rows that do not contain the bounding boxes for atypical,
# indeterminate or typical.
#df_train = df_train[(df_train["boxes"].apply(lambda x: bool(x))) | (df_train["label"] == "negative")]

df

In [ ]:
df_test = pd.read_csv(f"{root}/test.csv", index_col=0)
df_test

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.gradients = None
        
        # Convolutional layers
        self.conv = models.vgg16(pretrained=True).features
        
        # Freeze the parameters of the pretrained network except for the 
        # last few layers
        for i, param in enumerate(self.conv.parameters()):
            if i < 22:
                param.requires_grad = False
            
        # Fully connected layers
        self.dense = nn.Sequential(
            nn.Linear(25088, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 4),
            nn.Softmax()
        )
        
    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad
    
    # method for the gradient extraction
    def get_activations_gradient(self):
        return self.gradients
    
    # method for the activation exctraction
    def get_activations(self, x):
        return self.conv(x)

    def forward(self, x):
        x = self.conv(x)
        x.register_hook(self.activations_hook)
        x = self.dense(torch.flatten(x, 1))
        return x


model = Model()

for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
sum([p.numel() for p in model.parameters()])

In [ ]:
if not torch.cuda.is_available():  
    raise AssertionError("Turn on GPU")
print("Using GPU")

In [ ]:
class DataGenerator:
    def __init__(self, df, batch_size, im_shape, augment):
        self.df = df
        self.indexes = df.index.values
        self.df["code"] = self.df["label"].cat.codes
        self.batch_size = batch_size
        self.im_shape = im_shape
        self.augment = augment
        self.inputs = torch.empty((batch_size, *im_shape), dtype=torch.float32)
        self.targets = torch.empty(batch_size, dtype=torch.long)
        
    def get_image(self, image_id, label):
        filepath = f"{root}/train/{label}/{image_id}.png"
        with Image.open(filepath) as f:
            im = TF.to_tensor(f)  # (1, height, width)

        im = TF.resize(im, self.im_shape[1:])  # Resize
        
        if self.augment:
            # Affine transformations
            angle, translate, scale, shear = transforms.RandomAffine.get_params(
                [-30, 30], [0.05, 0.05], [0.8, 1.2], [-10, 10], im.shape[1:])
            im = TF.affine(im, angle, translate, scale, shear)

        im = im.repeat(3, 1, 1)  # Change grayscale to rgb by repeating the channel
        return im        
        
    def __iter__(self):
        """Initialize the iterator"""
        np.random.shuffle(self.indexes)  # Shuffle the input
        self.i = 0  # Index for going through the dataframe.
        
        # Move the tensors to GPU
        self.inputs = self.inputs.to("cuda:0")
        self.targets = self.targets.to("cuda:0")

        return self
    
    def __next__(self):
        while self.i < len(self):
            # Add an element to the batch.
            index = self.indexes[self.i]
            self.inputs[self.i % self.batch_size], self.targets[self.i % self.batch_size] = self[index]

            self.i += 1
            # If the batch is full, give it away
            if self.i % self.batch_size == 0:
                return self.inputs, self.targets

        # Move the tensors to back to CPU to release memory
        # TODO: Does this actually release memory from the GPU?
        self.inputs = self.inputs.to("cpu")
        self.targets = self.targets.to("cpu")

        raise StopIteration
        
    def __getitem__(self, index):
        """Get an input and its target."""
        row = self.df.loc[index]
        return self.get_image(row["image_id"], row["label"]), row["code"]
    
    def __len__(self):
        return len(self.df)

In [ ]:
# TESTING
data_generator = DataGenerator(df, 32, (3, 224, 224), augment=True)

fig, axs = plt.subplots(1, 5, figsize=(20, 10))

for i in range(5):
    im, target = data_generator[i]
    axs[i].imshow(im.permute(1, 2, 0), cmap=plt.cm.bone)
    axs[i].set_title(df["label"].cat.categories[target])
    
del data_generator

In [ ]:
# Initialize
batch_size_train = 32
batch_size_val = 1

indexes_train = df.sample(frac=0.9, random_state=42).index.values
indexes_val = df.drop(indexes_train).index.values
df_train = df.loc[indexes_train]
df_val = df.loc[indexes_val]

#min_count = df["label"].value_counts().min()
#n_samples_val = 20
#n_samples_train = min_count - n_samples_val
#df_train = pd.concat(df[df["label"] == label].sample(n_samples_train, random_state=42) for label in df["label"].cat.categories)
#df_rest = df.drop(df_train.index)
#df_val = pd.concat(df_rest[df_rest["label"] == label].sample(n_samples_val, random_state=42) for label in df["label"].cat.categories)

data_generator_train = DataGenerator(df_train, batch_size_train, (3, 224, 224), augment=True)
data_generator_val = DataGenerator(df_val, batch_size_val, (3, 224, 224), augment=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Train the model
epochs = 100

losses_train = []
losses_val = []
accuracies_val = []

n_iterations = epochs * (len(data_generator_train) // batch_size_train * batch_size_train + len(data_generator_val) // batch_size_val * batch_size_val)
progress_bar = tqdm(range(n_iterations))

model = model.to("cuda:0")

for epoch in range(epochs):
    model.train()  # Training mode
    losses_train.append([])  # Initialize list for losses for this epoch
    for inputs, targets in data_generator_train:
        optimizer.zero_grad()  # zero the parameter gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Calculate the loss
        loss.backward()  # Calculate the gradients
        optimizer.step()  # Optimize based on the gradients
        losses_train[-1].append(loss.item())  # Store the loss
        progress_bar.update(batch_size_train)

    model.eval()  # Set the model to evaluation mode.
    losses_val.append(0)  # Initialize the total loss for this epoch
    correct = 0
    for inputs_val, targets_val in data_generator_val:
        outputs_val = model(inputs_val)  # Forward pass
        losses_val[-1] += criterion(outputs_val, targets_val).item()  # Calculate the loss
        correct += outputs_val.max(1)[1][0].item() == targets_val.item()
        progress_bar.update(batch_size_val)
    
    # Calculate the metrics for this epoch
    loss_train_epoch = sum(losses_train[-1]) * batch_size_train / len(data_generator_train)
    loss_val_epoch = losses_val[-1] / len(data_generator_val)
    accuracy_epoch = correct / len(data_generator_val)
    accuracies_val.append(accuracy_epoch)
    
    # Save the model
    filepath = f"/kaggle/working/{epoch}.pt"
    torch.save(model.state_dict(), filepath)
        
    print(f"{epoch} Training loss: {loss_train_epoch}, Validation loss: {loss_val_epoch}, Validation accuracy: {accuracy_epoch}")
    
model = model.to("cpu")

print('Finished Training')

In [ ]:
# Load the best model
best_epoch = np.argmin([value / len(data_generator_val) for value in losses_val])
model.load_state_dict(torch.load(f"/kaggle/working/{best_epoch}.pt"))

In [ ]:
y_true = []
y_pred = []
tmp = []

model = model.to("cuda:0")
model.eval()  # Set the model to evaluation mode.

progress_bar = tqdm(range(len(data_generator_val)))
for inputs_val, targets_val in data_generator_val:
    outputs_val = model(inputs_val)
    tmp.append(outputs_val)
    y_true.append(targets_val.item())
    y_pred.append(outputs_val.max(1)[1][0].item())
    
    progress_bar.update(1)
    
cm = confusion_matrix(y_true, y_pred)

labels = df_train["label"].cat.categories.values
plt.matshow(cm, cmap=plt.cm.coolwarm)
plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)
#plt.tight_layout()
plt.ylabel("True")
plt.xlabel("Predicted")

for (i, j), z in np.ndenumerate(cm):
    plt.text(j, i, str(z), ha='center', va='center')